# Back testing Code

In [1]:
import pyupbit
import numpy as np
import time
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')
from tqdm import tqdm

In [2]:
coin_type = "KRW-BTC"
# OHLCV: Open High Low Close Volume. 당일 시가, 고가, 저가, 종가, 거래량에 대한 data
df = pyupbit.get_ohlcv(coin_type, count=230) # 7 days

In [3]:
# ==========================================================
# 전략: 변동성 돌파


# 변동폭*k 계산. (고가-저가)*k value
k = 0.032
df['range'] = (df['high'] - df['low']) * k

# target(매수가), range column을 한 칸씩 밑으로 내림
# 어제 가격을 오늘 반영해서 써야 하기 때문
df['target'] = df['open'] + df['range'].shift(1) 

# ==========================================================

In [4]:
# np.where(조건문, 참일때 값, 거짓일 때 값)
fee = 0.0005

# 수익률 = 목표가/종가 
df['ror'] = np.where(df['high'] > df['target'], df['close']/df['target'] - fee, 1)
df['ror']

2021-12-09 09:00:00    1.000000
2021-12-10 09:00:00    1.002061
2021-12-11 09:00:00    1.025996
2021-12-12 09:00:00    1.008768
2021-12-13 09:00:00    0.944190
                         ...   
2022-07-22 09:00:00    0.987210
2022-07-23 09:00:00    0.986304
2022-07-24 09:00:00    1.002195
2022-07-25 09:00:00    0.956427
2022-07-26 09:00:00    0.986143
Name: ror, Length: 230, dtype: float64

In [5]:
# 누적 곱 계산(cumprod) => 누적 수익률
df['hpr'] = df['ror'].cumprod()
df['hpr']

2021-12-09 09:00:00    1.000000
2021-12-10 09:00:00    1.002061
2021-12-11 09:00:00    1.028110
2021-12-12 09:00:00    1.037124
2021-12-13 09:00:00    0.979242
                         ...   
2022-07-22 09:00:00    0.442228
2022-07-23 09:00:00    0.436171
2022-07-24 09:00:00    0.437128
2022-07-25 09:00:00    0.418081
2022-07-26 09:00:00    0.412288
Name: hpr, Length: 230, dtype: float64

In [6]:
# Draw Down 계산. (누적 최대 값과 현재 hpr 차이/ 누적 최대값*100)
df['dd'] = (df['hpr'].cummax() - df['hpr']) / df['hpr'].cummax() *100
df

,open,high,low,close,volume,value,range,target,ror,hpr,dd
2021-12-09 09:00:00,62217000.0,62499000.0,59200000.0,59425000.0,6985.125259,4.250350e+11,105568.0,NaN,1.000000,1.000000,0.000000
2021-12-10 09:00:00,59425000.0,61995000.0,59106000.0,59683000.0,7746.325035,4.660654e+11,92448.0,59530568.0,1.002061,1.002061,0.000000
2021-12-11 09:00:00,59749000.0,61538000.0,59500000.0,61427000.0,3629.536770,2.197454e+11,65216.0,59841448.0,1.025996,1.028110,0.000000
2021-12-12 09:00:00,61414000.0,62887000.0,60904000.0,62049000.0,3665.345632,2.262398e+11,63456.0,61479216.0,1.008768,1.037124,0.000000
2021-12-13 09:00:00,61995000.0,62190000.0,58158000.0,58626000.0,7453.740910,4.474210e+11,129024.0,62058456.0,0.944190,0.979242,5.581005
...,...,...,...,...,...,...,...,...,...,...,...
2022-07-22 09:00:00,30415000.0,31201000.0,29874000.0,30080000.0,7607.908909,2.327444e+11,42464.0,30454296.0,0.987210,0.442228,57.360224
2022-07-23 09:00:00,30077000.0,30272000.0,29214000.0,29722000.0,5699.598361,1.699851e+11,33856.0,30119464.0,0.986304,0.436171,57.944229
2022-07-24 09:00:00,29704000.0,30195000.0,29498000.0,29818000.0,5186.691596,1.549393e+11,22304.0,29737856.0,1.002195,0.437128,57.851916
2022-07-25 09:00:00,29818000.0,29938000.0,28500000.0,28555000.0,8196.410471,2.386522e+11,46016.0,29840304.0,0.956427,0.418081,59.688424


In [7]:
print(f"MDD(%): {df['dd'].max()}") 

MDD(%): 63.79483150480583


In [8]:
df['hpr'][-1]

0.4122879543002408

# Back testing function

In [27]:
def back_testing(coin_type, k_value, term, prt:bool=False):
    time.sleep(0.1) # Redundant
    try:
        
        df = pyupbit.get_ohlcv(coin_type, count=term) # Redundant
        if type(df) == type(None):
            return 7210, 7210
        df['range'] = (df['high'] - df['low']) * k_value
        df['target'] = df['open'] + df['range'].shift(1) 

        fee = 0.0032
        df['ror'] = np.where(df['high'] > df['target'], df['close']/df['target'] - fee, 1)

        df['hpr'] = df['ror'].cumprod()
        df['dd'] = (df['hpr'].cummax() - df['hpr']) / df['hpr'].cummax() * 100

        pct = df['hpr'][-1]
        profit = round(pct-1, 5) * 100

        if (prt):
            print_str = f"{coin_type} {term}일간 수익률: {profit}%, MDD: {df['dd'].max()}"
            print(print_str)

    except Exception as e:
        print(e)
        time.sleep(0.1)
        return 7210, 7210
    
    return profit, df['dd'].max()

In [28]:
back_testing("KRW-ETH", 0.0032, 60, True)

KRW-ETH 60일간 수익률: -32.573%, MDD: 52.226943625968126


(-32.573, 52.226943625968126)

In [29]:
back_testing("KRW-ETH", 0.451, 60, True)

KRW-ETH 60일간 수익률: 16.487%, MDD: 11.128348857263422


(16.487, 11.128348857263422)

In [30]:
back_testing("KRW-BTC", 0.0032, 60, True)

KRW-BTC 60일간 수익률: -37.723%, MDD: 44.35628963887336


(-37.723, 44.35628963887336)

In [31]:
back_testing("KRW-BTC", 0.451, 60, True)

KRW-BTC 60일간 수익률: -14.325%, MDD: 19.461838130251312


(-14.325, 19.461838130251312)

# Finding Hyper parameter

In [22]:
def find_hyper_k(coin_type, term):
    # k value의 최적값을 찾기 위해 backtesting하며 수익률을 확인한다.
    df = pd.DataFrame([[0,0,0]], columns=['수익률', 'MDD%', 'k-value'])

    for i in tqdm(np.arange(0, 0.5, 0.001), desc='Progress', mininterval=0.1):
        profit, mdd = back_testing(coin_type, i, term, False)

        max_profit = df['수익률'].max()
        if profit >= max_profit:
            df = df.append(pd.Series([profit, mdd, i], index=df.columns), ignore_index=True)

    filter = df['수익률'] == df['수익률'].max()
    hyper_k = df[filter].iloc[0,2]
    
    return hyper_k

In [23]:
find_hyper_k("KRW-ETC", 14)

Progress:   0%|          | 1/500 [00:00<02:06,  3.95it/s]

'NoneType' object is not subscriptable


Progress:   0%|          | 2/500 [00:00<02:10,  3.81it/s]

'NoneType' object is not subscriptable


Progress:   1%|          | 3/500 [00:00<02:12,  3.76it/s]

...

Progress: 100%|█████████▉| 499/500 [02:08<00:00,  3.78it/s]

'NoneType' object is not subscriptable


Progress: 100%|██████████| 500/500 [02:09<00:00,  3.87it/s]

'NoneType' object is not subscriptable


0.0

In [25]:
find_hyper_k("KRW-KNC", 30)

Progress: 100%|██████████| 500/500 [01:22<00:00,  6.07it/s]


0

In [26]:
find_hyper_k("KRW-KNC", 14)

Progress: 100%|██████████| 500/500 [01:21<00:00,  6.11it/s]


0.34500000000000003

In [ ]:
df

In [20]:
# k value의 최적값을 찾기 위해 backtesting하며 수익률을 확인한다.
df = pd.DataFrame([[0,0,0]], columns=['수익률', 'MDD%', 'k-value'])

for i in tqdm(np.arange(0, 0.5, 0.001), desc='Progress', mininterval=0.1):
    profit, mdd = back_testing("KRW-KNC", i, 14, False)

    max_profit = df['수익률'].max()
    if profit >= max_profit:
        df = df.append(pd.Series([profit, mdd, i], index=df.columns), ignore_index=True)
filter = df['수익률'] == df['수익률'].max()
hyper_k = df[filter].iloc[0,2]

Progress: 100%|██████████| 500/500 [01:32<00:00,  5.38it/s]


In [21]:
filter = df['수익률'] == df['수익률'].max()
df[filter]

,수익률,MDD%,k-value
1,15.04,9.019347,0.0


In [36]:
def get_start_time(coin_type):
    df = pyupbit.get_ohlcv(coin_type, interval="day", count=1)
    start_time = df.index[0]
    return start_time
    
import datetime
now = datetime.datetime.now()
start_time = get_start_time(coin_type)
end_time = start_time + datetime.timedelta(days=1)

In [37]:
now

datetime.datetime(2022, 7, 26, 17, 20, 34, 700918)

In [38]:
start_time

Timestamp('2022-07-26 09:00:00')

In [39]:
end_time

Timestamp('2022-07-27 09:00:00')

In [40]:
start_time < now < end_time - datetime.timedelta(seconds=120)

True

In [41]:
pyupbit.get_current_price("KRW-BTC")

28174000.0